In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flg-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import flg_model
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy

fast_mode = False

In [2]:
all_data = fls.load_all_train_data()
np.random.default_rng(seed=0).shuffle(all_data)

# Pick N tomograms with 1 motor and N tomograms with 0 motors as validation set
N=100
n_motors = np.array([len(d.labels) for d in all_data])
inds_zero = np.argwhere(n_motors==0)[:N,0]
inds_one = np.argwhere(n_motors==1)[:N,0]
inds_test = np.concatenate((inds_zero,inds_one))
inds_train = np.setdiff1d(np.arange(len(n_motors)), inds_test)
inds_test.shape, inds_train.shape

train_data = []
for i in inds_train:
    train_data.append(all_data[i])
test_data = []
for i in inds_test:
    test_data.append(all_data[i])
np.random.default_rng(seed=0).shuffle(test_data)
test_data = test_data
if fast_mode:
    test_data = test_data[1:3]
len(train_data), len(test_data)

(444, 200)

In [3]:
[d.name for d in test_data]

['tomo_ff505c',
 'tomo_6c4df3',
 'tomo_62dbea',
 'tomo_fbb49b',
 'tomo_71d2c0',
 'tomo_6733fa',
 'tomo_c4db00',
 'tomo_9fc2b6',
 'tomo_13484c',
 'tomo_622ca9',
 'tomo_651ec2',
 'tomo_78b03d',
 'tomo_319f79',
 'tomo_518a1f',
 'tomo_cc65a9',
 'tomo_dbc66d',
 'tomo_399bd9',
 'tomo_94c173',
 'tomo_f1bf2f',
 'tomo_19a4fd',
 'tomo_d23087',
 'tomo_161683',
 'tomo_9cd09e',
 'tomo_6b1fd3',
 'tomo_c00ab5',
 'tomo_285454',
 'tomo_cf0875',
 'tomo_8f063a',
 'tomo_6c203d',
 'tomo_516cdd',
 'tomo_dee783',
 'tomo_ca8be0',
 'tomo_16efa8',
 'tomo_556257',
 'tomo_cd1a7c',
 'tomo_16fce8',
 'tomo_401341',
 'tomo_72b187',
 'tomo_997437',
 'tomo_0c2749',
 'tomo_db656f',
 'tomo_5b359d',
 'tomo_bb5ac1',
 'tomo_643b20',
 'tomo_22976c',
 'tomo_dae195',
 'tomo_df866a',
 'tomo_30b580',
 'tomo_82d780',
 'tomo_47d380',
 'tomo_95c0eb',
 'tomo_4ed9de',
 'tomo_ec1314',
 'tomo_134bb0',
 'tomo_dd36c9',
 'tomo_aff073',
 'tomo_cacb75',
 'tomo_4077d8',
 'tomo_24fda8',
 'tomo_a020d7',
 'tomo_1c38fd',
 'tomo_dfdc32',
 'tomo_8

In [ ]:
#importlib.reload(flg_unet)
N=5
if fast_mode: N=1
for ii in range(5):
    print('ii', ii)
    model = flg_model.ThreeStepModel()
    model.seed = ii
    #model.step1Heatmap.plot_every = 50
    if fast_mode: model.step1Heatmap.n_epochs = 3
    model.step2Labels.threshold = -2.
    model.train(train_data, test_data)
    print(' ')
    inferred_test_data = model.infer(test_data)
    print(fls.score_competition_metric(inferred_test_data, test_data))
    if not fast_mode:
        fls.dill_save(fls.model_dir + 'model_voxelbug' + str(ii) + '.pickle', (model, fls.score_competition_metric(inferred_test_data, test_data)))
    plt.title(str(ii))
    plt.pause(0.1)

ii 2
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

In [ ]:
model.infer(fls.load_all_test_data()[1:3])